# Auto-Indentation Prototype

This notebook prototypes the logic for the VisualGasic auto-formatter. 
We will implement a stack-based or counter-based indentation system for BASIC-like syntax.


In [ ]:

def auto_indent_visual_gasic(code: str) -> str:
    lines = code.split('\n')
    indented_lines = []
    current_indent = 0
    
    # 1. Define Rules
    indent_keywords = ["sub", "function", "for", "while", "do", "select case", "with"]
    dedent_keywords = ["end sub", "end function", "next", "wend", "loop", "end select", "end with", "end if"]
    
    # Special: "if ... then" (block) vs "if ... then ..." (single line)
    # Special: "else", "elseif", "case" -> temporary dedent for that line, but keeps block indent
    
    for line in lines:
        stripped = line.strip()
        lower = stripped.lower()
        
        # 2. Check Dedent (Apply before printing)
        should_dedent = False
        for k in dedent_keywords:
            if lower.startswith(k):
                should_dedent = True
                break
        
        # Handle Else/Case (dedent this line visually, but indent level stays high for next)
        is_mid_block = False
        if lower.startswith("else") or lower.startswith("elseif") or lower.startswith("case"):
            is_mid_block = True
            
        temp_indent = current_indent
        if should_dedent:
            current_indent -= 1
            if current_indent < 0: current_indent = 0
            temp_indent = current_indent
        elif is_mid_block:
            temp_indent = current_indent - 1
            if temp_indent < 0: temp_indent = 0

        # 3. Apply Indent
        tabs = "\t" * temp_indent
        indented_lines.append(tabs + stripped)
        
        # 4. Check Indent (Apply to NEXT line)
        should_indent = False
        
        # Block IF check
        if lower.startswith("if") and lower.endswith("then"):
            # Ensure it's not single line (in VB "If x Then y" handles itself, "If x Then" is block)
            # My current logic assumes "If ... Then" at end of line IS block start.
            should_indent = True
            
        if not should_indent:
            for k in indent_keywords:
                if lower.startswith(k):
                    # Special: "Do While" vs "Loop"
                    # If "Do" starts it, indent.
                    should_indent = True
                    break
        
        # Special Do/Loop handling
        if lower.startswith("loop"):
             # Loop usually ends a Do, so we dedented already.
             should_indent = False 
             
        if should_indent:
            current_indent += 1

    return "\n".join(indented_lines)

# Test
sample = """
Sub Main
Print "Start"
If 1=1 Then
Print "True"
For i=0 to 10
Print i
Next
Else
Print "False"
End If
End Sub
"""

print(auto_indent_visual_gasic(sample))
